# Дизайн хода остова белка

*весь код - для запуска в колабе. перед запуском включите gpu*

## 1. Постановка задачи

Возьмем структуру [альфа-конотоксина](https://www.rcsb.org/structure/1AKG) и попробуем задизайнить структуру белка, способного его связывать.

Загрузим файл со структурой и визуализируем ее.

In [ ]:
! wget https://files.rcsb.org/download/1AKG.pdb

In [ ]:
!pip install py3Dmol
!pip install Bio

from Bio import PDB
from Bio.PDB.PDBParser import PDBParser
import py3Dmol

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/1AKG.pdb','r').read(),'pdb')
p.setStyle({'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo() # центрировать структуру
p.show()

In [ ]:
parser=PDBParser()
structure=parser.get_structure("1AKG",'1AKG.pdb')
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.id[0]==' ':
                print(chain.id, residue.id[1], residue.get_resname())

## 2. Галлюцинации

Используем метод галлюцинаций RFDesign (https://github.com/RosettaCommons/RFDesign/tree/main/hallucination).

Данный метод использует модель для предсказания белковой структуры RosettaFold. Модель получает на вход случайную последовательность и попарную матрицу расстояний и с помощью множества шагов оптимизации (SGD и MCMC) минимизирует лосс-функцию. Также реализована ограниченная галлюцинация, когда на вход подается частично известная последовательность и/или структура и галлюцинируются недостающие части.

In [ ]:
# get repo and install dependencies
!git clone https://github.com/RosettaCommons/RFDesign.git
!pip -q install torch-geometric lie_learn icecream e3nn py3Dmol omegaconf
!pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html

# get weights
!wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_last.pt -P RFDesign/hallucination/weights/rf_Nov05/
!wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_mix_epoch25.pt -P RFDesign/inpainting/weights/

# get pyrosetta for sidechain relaxation
!gdown 1Pts0USd16GAtqz5v_4ErTu_7YBpysF9G
!pip install pyrosetta-2023.45+release.a6d9ba8-cp310-cp310-linux_x86_64.whl

In [ ]:
%%time
!mkdir toxin_hal
!python RFDesign/hallucination/hallucinate.py \
--pdb 1AKG.pdb --mask=80-100 --use_template no_contig \
--receptor 1AKG.pdb --steps=g100,m300 \
--w_rep 1 --w_atr 1 --rep_pdb 1AKG.pdb --atr_pdb 1AKG.pdb \
--num 1 --out toxin_hal/result


Посмотрим на сгенерированную последовательность

In [ ]:
with open('/content/toxin_hal/result_0.fas','r') as f:
    for line in f:
        print(line)

Сгенерированная структура исходно не имеет боковых цепей. Добавим их.

In [ ]:
%%time
! python RFDesign/scripts/RosettaTR/trfold_relax.py --roll -sg 7,3 toxin_hal/result_0.npz toxin_hal/result_0.pdb toxin_hal/result_0_relaxed.pdb

Визуализируем результат

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/toxin_hal/result_0_relaxed.pdb','r').read(),'pdb')
p.setStyle({'chain': 'A'},{'stick':{'colorscheme':'greenCarbon'}})
p.setStyle({'chain': 'B'},{'stick':{'colorscheme':'blueCarbon'}})

p.zoomTo() # центрировать структуру
p.show()

## 3. Диффузии

Используем диффузионную модель для генерации белка RFdiffusion (https://github.com/RosettaCommons/RFdiffusion)

Данный метод основан на модели денойзинга, которая на вход получает случайно распределенные координаты остатков и с помощью нескольких шагов денойзинга оптимизирует их координаты, получая осмысленную структуру белка. Как и в предыдущем случае, возможна частичная диффузия, когда часть структуры уже известна.

![diffusion](https://github.com/RosettaCommons/RFdiffusion/raw/main/img/main.png)

In [ ]:
!git clone https://github.com/RosettaCommons/RFdiffusion.git

!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/74f51cfb8b440f50d70878e05361d8f0/InpaintSeq_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/76d00716416567174cdb7ca96e208296/InpaintSeq_Fold_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt -P RFdiffusion/models -q
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/12fc204edeae5b57713c5ad7dcb97d39/Base_epoch8_ckpt.pt -P RFdiffusion/models -q

! pip install omegaconf hydra-core pyrsistent
!cd RFdiffusion/env/SE3Transformer && python setup.py install
!cd RFdiffusion && pip install -e .

Запустим RFDiffusion.

In [ ]:
%%time
! RFdiffusion/scripts/run_inference.py \
inference.input_pdb=1AKG.pdb \
'contigmap.contigs=[A1-16/0 80-100]' \
inference.output_prefix=toxin_diff/result inference.num_designs=1

Визуализируем результат

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/toxin_diff/result_0.pdb','r').read(),'pdb')
p.setStyle({'chain': 'A'},{'stick':{'colorscheme':'greenCarbon'}})
p.setStyle({'chain': 'B'},{'stick':{'colorscheme':'blueCarbon'}})
p.zoomTo()
p.show()

In [ ]:
parser=PDBParser(QUIET=True)
structure=parser.get_structure("diff",'/content/toxin_diff/result_0.pdb')
for model in structure:
    for chain in model:
        print(len(chain))
        print(' '.join([x.get_resname() for x in chain]))


Как можно заметить, модель сгенерировала структуру исключительно из глицинов. Что можно сделать в таком случае? Восстановить последовательность. Для предсказания последовательности по структуре существуют самые разные методы.

## 4. Использование галлюцинаций для восстановления последовательности

На прошлом занятии мы уже разбирали методы дизайна последовательности под нужную структуру с помошью стохастического градиентногого спуска. Похожим образом можно использовать и фреймворк для галлюцинаций, заставив его галлюцинировать только последовательность, но не структуру.


In [ ]:
%%time
!mkdir toxin_diff_hal
!python RFDesign/hallucination/hallucinate.py \
--pdb toxin_diff/result_0.pdb --mask= ... --use_template True \
--receptor 1AKG.pdb --steps=g60,m100 \
--spike 0.05 --w_crmsd 1 \
--num 1 --out toxin_diff_hal/result


In [ ]:
! python RFDesign/scripts/RosettaTR/trfold_relax.py --roll -sg 7,3 toxin_diff_hal/result_0.npz toxin_diff_hal/result_0.pdb toxin_diff_hal/result_0_relaxed.pdb

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/toxin_diff_hal/result_0_relaxed.pdb','r').read(),'pdb')
p.setStyle({'chain': 'A'},{'stick':{'colorscheme':'greenCarbon'}})
p.setStyle({'chain': 'B'},{'stick':{'colorscheme':'blueCarbon'}})
p.zoomTo() # центрировать структуру
p.show()

In [ ]:
with open('/content/toxin_diff_hal/result_0.fas','r') as f:
    for line in f:
        print(line)

## 5. Нейросети для предсказания последовательности

Для предсказания последовательности по структуре также существуют и обычные нейросети, например, ProteinMPNN (https://github.com/dauparas/ProteinMPNN). Возьмем фреймворк (https://github.com/nrbennet/dl_binder_design), включающий в себя предсказание последовательности и восстановление боковых цепей

In [ ]:
# get dl_binder_design to perform both sequence prediction and sidechain relaxation
!git clone https://github.com/nrbennet/dl_binder_design.git
!cd dl_binder_design/mpnn_fr; git clone https://github.com/dauparas/ProteinMPNN.git

In [ ]:
%%time
!python dl_binder_design/helper_scripts/addFIXEDlabels.py --pdbdir toxin_diff --trbdir toxin_diff --verbose
!python dl_binder_design/mpnn_fr/dl_interface_design.py  -pdbdir toxin_diff  -outpdbdir toxin_diff_mpnn \
 -seqs_per_struct 1 -relax_cycles 2


In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/toxin_diff_mpnn/result_0_dldesign_0_cycle2.pdb','r').read(),'pdb')
p.setStyle({'chain': 'A'},{'stick':{'colorscheme':'greenCarbon'}})
p.setStyle({'chain': 'B'},{'stick':{'colorscheme':'blueCarbon'}})
p.zoomTo() # центрировать структуру
p.show()

In [ ]:
parser=PDBParser(QUIET=True)
structure=parser.get_structure("1AKG",'toxin_diff_mpnn/result_0_dldesign_0_cycle2.pdb')
for model in structure:
    for chain in model:
        print(len(chain))
        print(' '.join([x.get_resname() for x in chain]))

## 6. Домашняя работа

1. Предскажите структуры сгенерированных последовательностей с помощью альфафолда, рассчитайте их rmsd со структурами, сгенерированными rfdesign и rfdiffusion, сделайте вывод о качестве предсказания.
2. Рассчитайте внутреннюю энергию полученных комплексов, сделайте вывод о стабильности сгенерированных белков.

*оба задания можно сделать с использованием функционала RFDesign (подсказки здесь https://github.com/RosettaCommons/RFDesign/blob/main/tutorials/halluc_PD-L1_binder/halluc_PD-L1_binder.ipynb)*